# Assignment 4 - Hypothesis Testing


Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.




### Question 1
Returns a DataFrame of towns and the states they are in from the university_towns.txt list. The format of the DataFrame should be:

DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
columns=["State", "RegionName"]  )
    
The following cleaning needs to be done:

  1. For "State", removing characters from "[" to the end.
  2. For "RegionName", when applicable, removing every character from " (" to the end.
  3. Depending on how you read the data, you may need to remove newline character

In [5]:
!pip install xlrd

import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from scipy import stats

     |████████████████████████████████| 112kB 30.7MB/s eta 0:00:01


In [6]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [7]:
university_towns = pd.DataFrame()

with open('university_towns.txt', 'r') as file:  
        for line in file:
            if '[edit]' in line:
                State = line[:line.find('[')]
                State = State.strip()
                continue
            Region = line.strip()
            if '(' in Region:
                Region = Region[:(Region.find('('))]
                Region = Region.strip()
                university_towns = university_towns.append(pd.DataFrame([[State, Region]], columns=['State', 'RegionName']), ignore_index=True)

university_towns.head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


### Question 2

Return the year and quarter of the recession start time as a string value in a format such as 2005q3.

In [9]:
df = pd.read_excel('gdplev.xls', skiprows= 219)
col_to_keep = ['1999q4', 12323.3]
df = df[col_to_keep]
df = df.rename(columns={'1999q4': 'Quarter', 12323.3: 'GDP'})
df.head()

,Quarter,GDP
0,2000q1,12359.1
1,2000q2,12592.5
2,2000q3,12607.7
3,2000q4,12679.3
4,2001q1,12643.3


In [10]:
recession_start = []

for i in range(len(df)):
    if (df.iloc[i]['GDP'] < df.iloc[i - 1]['GDP']) and (df.iloc[i]['GDP'] > df.iloc[i + 1]['GDP']):
        recession_start.append(df.iloc[i])

start_q = recession_start[0].Quarter
start_q

'2008q3'

### Question 3

Return the year and quarter of the recession end time as a string value in a format such as 2005q3.

In [11]:
indx_start = df[df.Quarter == recession_start[0].Quarter] 
indx_start = indx_start.index.tolist()
indx_start = indx_start[0]

recession_end = []

for i in range (indx_start, len(df)):
    if (df.iloc[i]['GDP'] > df.iloc[i - 1]['GDP']) and (df.iloc[i-1]['GDP'] > df.iloc[i - 2]['GDP']):
        recession_end.append(df.iloc[i])

end_q = recession_end[0].Quarter
end_q

'2009q4'

### Question 4

Return the year and quarter of the recession bottom time as a string value in a format such as 2005q3.

In [14]:
indx_end = df[df.Quarter == recession_end[0].Quarter] 
indx_end = indx_end.index.tolist()
indx_end = indx_end[0]

rec_period = df.loc[indx_start:indx_end]
rec_period = rec_period[rec_period["GDP"] == rec_period.GDP.min()]

#getting the index
rec_bottom = rec_period.index.tolist()
rec_bottom = rec_bottom[0]

bottom_q = rec_period.Quarter.values[0]
bottom_q

'2009q2'

### Question 5

Converts the housing data to quarters and returns it as mean values in a dataframe. This dataframe should be a dataframe with columns for 2000q1 through 2016q3, and should have a multi-index in the shape of ["State","RegionName"].
    
Note: Quarters are defined in the assignment description, they are not arbitrary three month periods.
    
The resulting dataframe should have 67 columns, and 10,730 rows.

In [15]:
def get_quarter(year, month):
    if month <= 3:
        quarter = 1
    elif month <= 6:
        quarter = 2
    elif month <= 9:
        quarter = 3
    elif month <= 12:
        quarter = 4
    return (str(year) + 'q' + str(quarter))

In [16]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

GDPq = pd.read_csv('City_Zhvi_AllHomes.csv')
GDPq = (GDPq.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1).replace(to_replace='NaN', value=np.NaN))
GDPq = GDPq.replace({'State': states})
GDPq = GDPq.set_index(['State', 'RegionName'])
GDPq = GDPq.loc[:, '2000-01':] 

In [17]:
limit = len(GDPq.columns)
i = 0
while i < limit:
    col_name = GDPq.iloc[:, i].name
    year = int(col_name.split('-')[0])
    month = int(col_name.split('-')[1])
    quarter = get_quarter(year, month)
    if i + 3 < limit:
        newcols = GDPq.iloc[:, i:i + 3]
    else:
        newcols = GDPq.iloc[:, i:limit]
    GDPq[quarter] = newcols.mean(axis=1)
    i += 3

In [18]:
GDPq = GDPq.drop(GDPq.columns[:limit], axis=1)
GDPq.head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,2002q3,2002q4,2003q1,2003q2,2003q3,2003q4,2004q1,2004q2,2004q3,2004q4,2005q1,2005q2,2005q3,2005q4,2006q1,2006q2,2006q3,2006q4,2007q1,2007q2,2007q3,2007q4,2008q1,2008q2,2008q3,2008q4,2009q1,2009q2,2009q3,2009q4,2010q1,2010q2,2010q3,2010q4,2011q1,2011q2,2011q3,2011q4,2012q1,2012q2,2012q3,2012q4,2013q1,2013q2,2013q3,2013q4,2014q1,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435650.000000,432133.333333,422666.666667,431800.000000,438766.666667,464500.000000,488133.333333,513700.000000,520033.333333,524700.000000,517233.333333,514666.666667,510200.000000,513366.666667,508500.000000,503933.333333,499766.666667,487933.333333,477733.333333,465833.333333,455933.333333,458366.666667,469700.000000,476933.333333,478566.666667,477266.666667,476700.000000,472133.333333,469200.000000,463700.000000,460200.000000,462700.000000,467233.333333,469166.666667,469033.333333,472233.333333,481366.666667,492433.333333,503133.333333,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,287000.000000,302233.333333,318900.000000,334500.000000,350633.333333,370833.333333,395233.333333,425566.666667,456166.666667,477366.666667,495066.666667,518400.000000,545566.666667,571066.666667,581766.666667,587500.000000,592966.666667,592533.333333,591766.666667,589500.000000,577300.000000,561366.666667,535300.000000,502266.666667,469500.000000,443966.666667,426266.666667,413900.000000,406366.666667,404333.333333,413366.666667,411166.666667,406766.666667,402066.666667,397100.000000,390100.000000,382100.000000,375566.666667,370600.000000,375333.333333,385566.666667,396566.666667,411533.333333,432866.666667,455066.666667,471933.333333,484266.666667,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,178966.666667,183700.000000,187100.000000,194300.000000,202066.666667,204133.333333,206200.000000,208600.000000,212766.666667,218333.333333,223900.000000,228533.333333,233166.666667,237500.000000,242400.000000,245200.000000,245700.000000,246700.000000,247233.333333,246400.000000,246266.666667,246833.333333,243733.333333,237900.000000,232000.000000,227033.333333,223766.666667,219700.000000,214100.000000,211666.666667,207300.000000,203800.000000,198933.333333,191366.666667,187400.000000,181600.000000,174266.666667,170200.000000,166933.333333,167033.333333,168133.333333,167233.333333,168366.666667,172633.333333,180800.000000,185866.666667,190900.000000,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,62166.666667,63500.000000,64733.333333,66200.000000,68466.666667,70733.333333,73733.333333,76733.333333,80566.666667,85533.333333,89700.000000,94866.666667,100166.666667,104900.000000,108333.333333,112366.666667,115466.666667,116633.333333,117333.333333,118700.000000,120833.333333,120800.000000,119566.666667,118133.333333,116933.333333,115866.666667,116200.000000,116166.666667,116733.333333,118566.666667,121166.666667,122800.000000,120166.666667,117766.666667,117100.000000,116300.000000,115466.666667,114700.000000,112733.333333,110533.333333,109933.333333,110833.333333,112000.000000,111733.333333,111000.0

In [19]:
GDP_new = GDPq.copy()
start_index = indx_start
bottom_index = rec_bottom
GDP_new['Ratio'] = GDP_new.apply(lambda x: x[start_q]/x[bottom_q], axis=1)
GDP_new = GDP_new[['Ratio']]
GDP_new.head()

,,Ratio
State,RegionName,
New York,New York,1.072844
California,Los Angeles,1.134332
Illinois,Chicago,1.055985
Pennsylvania,Philadelphia,1.006600
Arizona,Phoenix,1.151773


In [20]:
uni_towns = university_towns.copy()
uni_towns = uni_towns.set_index(['State', 'RegionName'])
print (uni_towns.shape)

uni_town_price = pd.merge(GDP_new, uni_towns, how="inner", left_index=True, right_index=True).dropna()
print (uni_town_price.shape)

non_uni_town_price = pd.merge(GDP_new, uni_towns, how="left", left_index=True, right_index=True).dropna()
print (non_uni_town_price.shape)

p = stats.ttest_ind(uni_town_price['Ratio'], non_uni_town_price['Ratio'])

(513, 0)
(256, 1)
(9884, 1)


In [21]:
if p[1] < 0.01:
    different = True
else:
    different = False
    
if uni_town_price['Ratio'].mean() < non_uni_town_price['Ratio'].mean():
    better = "University Town"
else:
    better = "Non-University Town"
    
(p[1], different, better)

(0.005437515950071544, True, 'University Town')